<a href="https://colab.research.google.com/github/Rizzwick/week_5/blob/main/week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
z!pip install pathway panel bokeh geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 19

In [ ]:
import pathway as pw
import panel as pn
import bokeh.plotting
import bokeh.models
import pandas as pd
import numpy as np
from geopy.distance import geodesic

pn.extension('bokeh')

In [ ]:
fname = "/content/dataset.csv"
df = pd.read_csv(fname)
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

In [ ]:
BASE_PRICE = 10
ALPHA = 2
DM_ALPHA = 1.0
DM_BETA = 0.5
DM_GAMMA = 0.3
DM_DELTA = 0.7
DM_EPSILON = 0.2
DM_LAMBDA = 0.7

vehicle_type_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}

In [ ]:
BASE_PRICE = 10
ALPHA = 2
DM_ALPHA = 1.0
DM_BETA = 0.5
DM_GAMMA = 0.3
DM_DELTA = 0.7
DM_EPSILON = 0.2
DM_LAMBDA = 0.7

vehicle_type_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}

from geopy.distance import geodesic

def compute_demand(row):
    vtype_weight = vehicle_type_map.get(row['VehicleType'], 1.0)
    traffic_val = 1 if str(row['TrafficConditionNearby']).lower() == 'high' else 0
    return (
        DM_ALPHA * (row['Occupancy'] / row['Capacity']) +
        DM_BETA * row['QueueLength'] -
        DM_GAMMA * traffic_val +
        DM_DELTA * row['IsSpecialDay'] +
        DM_EPSILON * vtype_weight
    )

def demand_based_price(norm_demand):
    price = BASE_PRICE * (1 + DM_LAMBDA * norm_demand)
    return np.clip(price, 0.5 * BASE_PRICE, 2 * BASE_PRICE)

def get_nearby_lots(lot_id, lat, lon, df, radius=0.5):
    lots = []
    for idx, row in df[df['SystemCodeNumber'] != lot_id][['SystemCodeNumber', 'Latitude', 'Longitude']].drop_duplicates().iterrows():
        dist = geodesic((lat, lon), (row['Latitude'], row['Longitude'])).km
        if dist <= radius:
            lots.append(row['SystemCodeNumber'])
    return lots

def competitive_price(row, df):
    lot_id = row['SystemCodeNumber']
    lat, lon = row['Latitude'], row['Longitude']
    nearby_lots = get_nearby_lots(lot_id, lat, lon, df)
    competitor_prices = df[(df['SystemCodeNumber'].isin(nearby_lots)) & (df['Timestamp'] == row['Timestamp'])]['DemandPrice']
    if not competitor_prices.empty:
        avg_competitor_price = competitor_prices.mean()
        if row['Occupancy'] >= row['Capacity']:
            if avg_competitor_price < row['DemandPrice']:
                return avg_competitor_price * 0.95
        elif avg_competitor_price > row['DemandPrice']:
            return min(row['DemandPrice'] * 1.05, 2 * BASE_PRICE)
    return row['DemandPrice']


In [ ]:
schema = pw.schema_from_csv(fname)
data = pw.demo.replay_csv(fname, schema=schema, input_rate=1000)

In [ ]:
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['baseline_price'] = BASE_PRICE + ALPHA * df['OccupancyRate']

df['Demand'] = df.apply(compute_demand, axis=1)

demand_min, demand_max = df['Demand'].min(), df['Demand'].max()
if demand_max - demand_min == 0:
    df['NormDemand'] = 0.5
else:
    df['NormDemand'] = (df['Demand'] - demand_min) / (demand_max - demand_min)

df['DemandPrice'] = df['NormDemand'].apply(demand_based_price)


In [ ]:
sample_df = df.sample(n=500, random_state=42).copy()
sample_df['CompetitivePrice'] = sample_df.apply(lambda row: competitive_price(row, sample_df), axis=1)


In [ ]:
def price_plotter(src):
    fig = bokeh.plotting.figure(
        height=400,
        width=700,
        title="Real-Time Parking Lot Pricing",
        x_axis_type="datetime"
    )
    fig.line("Timestamp", "baseline_price", source=src, line_width=2, color="blue", legend_label="Baseline")
    fig.line("Timestamp", "DemandPrice", source=src, line_width=2, color="green", legend_label="Demand-Based")
    fig.line("Timestamp", "CompetitivePrice", source=src, line_width=2, color="red", legend_label="Competitive")
    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"
    fig.legend.location = "top_left"
    return fig

In [ ]:
lot_id = 'BHMBCCMKT01'
lot_data = sample_df[sample_df['SystemCodeNumber'] == lot_id].sort_values('Timestamp')
src = bokeh.models.ColumnDataSource(lot_data)

dashboard = pn.Row(
    price_plotter(src),
    pn.widgets.DataFrame(lot_data[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'baseline_price', 'DemandPrice', 'CompetitivePrice']])
)

dashboard.servable()


Row
    [0] Bokeh(figure)
    [1] DataFrame(value=     SystemCodeNumber  ...)